In [3]:
%matplotlib inline
#basic python packages for plotting and array management
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
#import statsmodels.api as sm
from datetime import datetime

#for data import
import pandas as pd
from __future__ import division
import os
import sys
from importlib import reload
reload(sys)

#Preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

#Classifiers
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV, PassiveAggressiveClassifier, Perceptron, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier

#Ensemble methods and crossvalidation
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier,RandomForestClassifier, VotingClassifier

#PostProcessing
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import time

In [4]:
customers = pd.read_csv("customers6.csv")
products = pd.read_csv("products.csv")
X_train = pd.read_csv("X_train.csv")
X_test   = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")

# CUSTOMERS

In [5]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate
0,17860216.0,GB,22/10/2015 15:00,Femme,17/10/2015 12:58
1,10588204.0,FR,20/10/2015 19:46,Femme,20/10/2015 00:07
2,17841489.0,DE,20/10/2015 16:40,Homme,12/10/2015 23:04
3,17849688.0,DE,15/10/2015 14:17,Homme,22/10/2015 10:41
4,12916644.0,FR,13/10/2015 15:54,Femme,15/03/2012 10:04


### BirthDate -> Age

In [6]:
customers['BirthDate'] = customers['BirthDate'].apply(lambda x : datetime.strptime(x, '%d/%m/%Y %H:%M') )
customers['Age'] = 2018 - customers['BirthDate'].apply(lambda x : x.year)

In [7]:
for i in range(10,60,5):
    customers['Age' + str(i) + '-' + str(i+5)] = ((customers['Age']>i) & (customers['Age']<=i+5)).astype('float')

In [8]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate,Age,Age10-15,Age15-20,Age20-25,Age25-30,Age30-35,Age35-40,Age40-45,Age45-50,Age50-55,Age55-60
0,17860216.0,GB,2015-10-22 15:00:00,Femme,17/10/2015 12:58,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10588204.0,FR,2015-10-20 19:46:00,Femme,20/10/2015 00:07,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17841489.0,DE,2015-10-20 16:40:00,Homme,12/10/2015 23:04,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17849688.0,DE,2015-10-15 14:17:00,Homme,22/10/2015 10:41,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12916644.0,FR,2015-10-13 15:54:00,Femme,15/03/2012 10:04,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Gender -> Binaire

In [9]:
## 1 si une femme
customers['Sexe'] = customers['Gender'].eq('Femme').astype('float')

### FirstOrderDate -> Time in Year since first Order

In [10]:
customers['FirstOrderDate'] = customers['FirstOrderDate'].apply(lambda x : datetime.strptime(x, '%d/%m/%Y %H:%M') )
customers['FirstOrder'] = 2018 - customers['FirstOrderDate'].apply(lambda x : x.year)

In [11]:
for i in range(3,10):
    customers['FirstOrder' + '-' + str(i) + 'Y'] = (customers['FirstOrder']==i).astype('float')
customers['FirstOrder-10Y+'] = (customers['FirstOrder']>=10).astype('float')

In [12]:
def creation(Table,Colonne,Liste):
    for element in Liste:     # On parcourt la liste en entier
        names = []
        for nuance in element:    # On parcourt chaque nuance de chaque sous-liste
            name_col = nuance
            name = Table[Colonne].str.contains(nuance).astype('float')
            names.append(name)
        
        col = pd.Series(0,index = range(Table.shape[0]))
        for column in names :
            col = col+column
        
        Table[name_col] = col
    return ('%d colonne(s) ont étaient ajoutées à la DataFrame' %(len(Liste)))

In [13]:
customers.head()

,CustomerId,CountryISOCode,BirthDate,Gender,FirstOrderDate,Age,Age10-15,Age15-20,Age20-25,Age25-30,...,Sexe,FirstOrder,FirstOrder-3Y,FirstOrder-4Y,FirstOrder-5Y,FirstOrder-6Y,FirstOrder-7Y,FirstOrder-8Y,FirstOrder-9Y,FirstOrder-10Y+
0,17860216.0,GB,2015-10-22 15:00:00,Femme,2015-10-17 12:58:00,3,0.0,0.0,0.0,0.0,...,1.0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10588204.0,FR,2015-10-20 19:46:00,Femme,2015-10-20 00:07:00,3,0.0,0.0,0.0,0.0,...,1.0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17841489.0,DE,2015-10-20 16:40:00,Homme,2015-10-12 23:04:00,3,0.0,0.0,0.0,0.0,...,0.0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17849688.0,DE,2015-10-15 14:17:00,Homme,2015-10-22 10:41:00,3,0.0,0.0,0.0,0.0,...,0.0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12916644.0,FR,2015-10-13 15:54:00,Femme,2012-03-15 10:04:00,3,0.0,0.0,0.0,0.0,...,1.0,6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [14]:
customers.dtypes

CustomerId                float64
CountryISOCode             object
BirthDate          datetime64[ns]
Gender                     object
FirstOrderDate     datetime64[ns]
Age                         int64
Age10-15                  float64
Age15-20                  float64
Age20-25                  float64
Age25-30                  float64
Age30-35                  float64
Age35-40                  float64
Age40-45                  float64
Age45-50                  float64
Age50-55                  float64
Age55-60                  float64
Sexe                      float64
FirstOrder                  int64
FirstOrder-3Y             float64
FirstOrder-4Y             float64
FirstOrder-5Y             float64
FirstOrder-6Y             float64
FirstOrder-7Y             float64
FirstOrder-8Y             float64
FirstOrder-9Y             float64
FirstOrder-10Y+           float64
dtype: object

####  Delete columns

In [15]:
del customers['Gender']
del customers['FirstOrderDate']
del customers['FirstOrder']
del customers['BirthDate']
del customers['Age']

In [16]:
customers = customers.fillna(0)

In [17]:
customers.head()

,CustomerId,CountryISOCode,Age10-15,Age15-20,Age20-25,Age25-30,Age30-35,Age35-40,Age40-45,Age45-50,...,Age55-60,Sexe,FirstOrder-3Y,FirstOrder-4Y,FirstOrder-5Y,FirstOrder-6Y,FirstOrder-7Y,FirstOrder-8Y,FirstOrder-9Y,FirstOrder-10Y+
0,17860216.0,GB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10588204.0,FR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17841489.0,DE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17849688.0,DE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12916644.0,FR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


# ORDERS

In [18]:
orders = X_train

### LineItem -> LineItem binary position

In [19]:
for i in range(1,5):
    orders['LineItem' + '_' + str(i) ] = (orders['LineItem']==i).astype('float')
orders['LineItem_5+'] = (orders['LineItem']>=5).astype('float')

### OrderStatusLabel -> Binary (Expedie = 1)

In [20]:
orders['OrderStatus'] = orders['OrderStatusLabel'].eq('Expédié').astype('float')

### OrderTypelabel -> Binary (DIRECT = 1)

In [21]:
orders['OrderType'] = orders['OrderTypelabel'].eq('DIRECT').astype('float')

### SeasonLabel -> Binary (Automne/Hiver = 1)

In [22]:
orders['Season'] = orders['SeasonLabel'].eq('Automne/Hiver').astype('float')

### PayementModeLabel -> Binary Payement Mode

In [23]:
Label = [['Carte','maestro'],['PayPal'],['Gratuit'],['iDeal'],['que'],['Bank'],['DHL']]
creation(orders,'PayementModeLabel',Label)

'7 colonne(s) ont étaient ajoutées à la DataFrame'

### CustomerTypeLabel -> Binary Type (Fidelise = 1)

In [24]:
orders['CustomerType'] = orders['CustomerTypeLabel'].eq('Fidélisé').astype('float')

### IsoCode : Inchangé

### DeviceTypeLabel  (attention, ND majoritaire)

In [25]:
Label2 = [['Desktop'],['Tablet'],['Mobile']]
creation(orders,'DeviceTypeLabel',Label2)

'3 colonne(s) ont étaient ajoutées à la DataFrame'

### PricingTypeLabel -> Binary Type

In [26]:
Label3 = [['Plein'],['Soldes'],['Avec CP'],['Sans CP'],['Vente Priv']]
creation(orders,'PricingTypeLabel',Label3)

'5 colonne(s) ont étaient ajoutées à la DataFrame'

### TotalLineItems -> Same as LineItem

In [27]:
for i in range(1,6):
    orders['TotalLineItems' + '_' + str(i) ] = (orders['TotalLineItems']==i).astype('float')
orders['TotalLineItems_6+'] = (orders['TotalLineItems']>=6).astype('float')

### OrderCreationDate -> Categories based on order creation hour

In [28]:
orders['OrderCreationDate'] = orders['OrderCreationDate'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )

In [29]:
temp = orders['OrderCreationDate'].apply(lambda x : x.hour)

orders['Hour_21H_1H'] = ((temp<=1) | (temp>=21)).astype('float')

### OrderShipDate -> on pense que la date d'envoi ne joue pas dans le retour du produit

In [30]:
orders.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,...,Avec CP,Sans CP,Vente Priv,TotalLineItems_1,TotalLineItems_2,TotalLineItems_3,TotalLineItems_4,TotalLineItems_5,TotalLineItems_6+,Hour_21H_1H
0,73521754,439729,1,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,73521754,440174,2,12443972,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,73525226,494501,1,12443958,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,73529009,439590,1,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,73529009,559476,2,12443946,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [31]:
orders.columns

Index(['OrderNumber', 'VariantId', 'LineItem', 'CustomerId',
       'OrderStatusLabel', 'OrderTypelabel', 'SeasonLabel',
       'PayementModeLabel', 'CustomerTypeLabel', 'IsoCode', 'DeviceTypeLabel',
       'PricingTypeLabel', 'TotalLineItems', 'Quantity', 'UnitPMPEUR',
       'OrderCreationDate', 'OrderShipDate', 'OrderNumCustomer', 'IsOnSale',
       'BillingPostalCode', 'LineItem_1', 'LineItem_2', 'LineItem_3',
       'LineItem_4', 'LineItem_5+', 'OrderStatus', 'OrderType', 'Season',
       'maestro', 'PayPal', 'Gratuit', 'iDeal', 'que', 'Bank', 'DHL',
       'CustomerType', 'Desktop', 'Tablet', 'Mobile', 'Plein', 'Soldes',
       'Avec CP', 'Sans CP', 'Vente Priv', 'TotalLineItems_1',
       'TotalLineItems_2', 'TotalLineItems_3', 'TotalLineItems_4',
       'TotalLineItems_5', 'TotalLineItems_6+', 'Hour_21H_1H'],
      dtype='object')

In [32]:
# delete columns 
del orders['OrderNumber']
#del orders['VariantId'] # Necssaire pour merge orders/products
del orders['LineItem']
#del orders['CustomerId'] # Necessaire pour merge orders/customers
del orders['OrderStatusLabel']
del orders['OrderTypelabel']
del orders['SeasonLabel']
del orders['PayementModeLabel']
del orders['CustomerTypeLabel']
del orders['DeviceTypeLabel']
del orders['PricingTypeLabel']
del orders['TotalLineItems']
del orders['OrderCreationDate']
del orders['OrderShipDate']
del orders['BillingPostalCode']
# del orders['OrderNumCustomer']  # Le garder et voir statistiques dessus, pas traité.

In [33]:
orders = orders.fillna(0)

In [34]:
orders.head()

,VariantId,CustomerId,IsoCode,Quantity,UnitPMPEUR,OrderNumCustomer,IsOnSale,LineItem_1,LineItem_2,LineItem_3,...,Avec CP,Sans CP,Vente Priv,TotalLineItems_1,TotalLineItems_2,TotalLineItems_3,TotalLineItems_4,TotalLineItems_5,TotalLineItems_6+,Hour_21H_1H
0,439729,12443972,FR,1,"52,64",1,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,440174,12443972,FR,1,"52,64",1,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,494501,12443958,FR,1,"13,17",1,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,439590,12443946,FR,1,"56,4",1,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,559476,12443946,FR,1,37,1,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


# Products

In [35]:
products.head()

,VariantId,GenderLabel,MarketTargetLabel,SeasonLabel,SeasonalityLabel,BrandId,UniverseLabel,TypeBrand,ProductId,ProductType,...,UpperHeight,HeelHeight,PurchasePriceHT,IsNewCollection,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription
0,728257.0,Homme,Classique,Automne/Hiver,Saisonnier,66.0,DÃ©tente,Standard,17267.0,Baskets,...,NaN,0.0,30.50,0.0,Montantes,NaN,NaN,NaN,False,Prenez une taille en dessous de votre pointure...
1,806356.0,Femme,ND,Automne/Hiver,Saisonnier,842.0,ND,Standard,30824.0,Baskets,...,0.0,0.0,43.00,0.0,Montantes,NaN,NaN,NaN,True,Prenez votre pointure habituelle
2,768790.0,Femme,ND,Automne/Hiver,Reconduit,988.0,Ville,Standard,62475.0,Bottines et boots,...,6.0,3.0,54.90,0.0,Bout pointu,NaN,NaN,NaN,False,Prenez votre pointure habituelle
3,515679.0,Femme,ND,Automne/Hiver,Saisonnier,769.0,Ville,Standard,43983.0,Escarpins,...,0.0,13.0,34.50,0.0,Bout rond,NaN,NaN,NaN,False,Prenez votre pointure habituelle
4,1025246.0,Femme,ND,Automne/Hiver,Saisonnier,1244.0,ND,Standard,81493.0,Bottines et boots,...,8.0,4.0,43.76,0.0,Bout rond,NaN,NaN,NaN,False,Prenez votre pointure habituelle


#### 1) Fonction qui crée les colonnes nécessaires

In [36]:
def creation(Table,Colonne,Liste):
    for element in Liste:     # On parcourt la liste en entier
        names = []
        for nuance in element:    # On parcourt chaque nuance de chaque sous-liste
            name_col = nuance
            name = Table[Colonne].str.contains(nuance).astype('float')
            names.append(name)
        
        col = pd.Series(0,index = range(Table.shape[0]))
        for column in names :
            col = col+column
        
        Table[name_col] = col
    return ('%d colonne(s) ont étaient ajoutées à la DataFrame' %(len(Liste)))

#### 2) Analyse de la table

In [37]:
products.dtypes

VariantId                float64
GenderLabel               object
MarketTargetLabel         object
SeasonLabel               object
SeasonalityLabel          object
BrandId                  float64
UniverseLabel             object
TypeBrand                 object
ProductId                float64
ProductType               object
SupplierColor             object
ProductColorId           float64
MinSize                  float64
MaxSize                  float64
CalfTurn                 float64
UpperHeight              float64
HeelHeight               float64
PurchasePriceHT          float64
IsNewCollection          float64
SubtypeLabel              object
UpperMaterialLabel        object
LiningMaterialLabel       object
OutSoleMaterialLabel      object
RemovableSole             object
SizeAdviceDescription     object
dtype: object

#### 3) Modification+ajout de colonnes

##### * Gender Label

In [38]:
Label = [['Femme'],['Enfant'],['Homme'],['Sacs'],['Accessoires']]
creation(products,'GenderLabel',Label)

'5 colonne(s) ont étaient ajoutées à la DataFrame'

##### *MarketTargetLabel

In [39]:
Label2 = [['Contemporaine','contemporain'],['Suiveur'],['Fashionistas'],['Classique'],['Confort'],['Baroudeur','Aventurier'],['Skate']]
creation(products,'MarketTargetLabel',Label2)


'7 colonne(s) ont étaient ajoutées à la DataFrame'

##### *SeasonLabel

In [40]:
Label3 = products['SeasonLabel'].eq('Automne/Hiver').astype('float')
products['SeasonLabel'] = Label3

##### *SeasonalityLabel

In [41]:
Label4 = [['Saisonnier'],['Reconduit'],['Permanent']]
creation(products,'SeasonalityLabel',Label4)

'3 colonne(s) ont étaient ajoutées à la DataFrame'

##### *BrandId

In [42]:
## Problème car représenté par des nombres donc ordonés mais il y en a trop

##### *UniverseLabel

In [43]:
Label5 = [['tente'],['Ville'],['Sport'],['Green'],['Luxe'],['Confort'],['Soir']]
creation(products,'UniverseLabel',Label5)

'7 colonne(s) ont étaient ajoutées à la DataFrame'

##### *TypeBrand

In [44]:
Label6 = [['Standard'],['Luxe'],['Super']]
creation(products,'TypeBrand',Label6)

'3 colonne(s) ont étaient ajoutées à la DataFrame'

##### *ProductType

In [45]:
Label7 = [['Baskets'],['Sandales'],['Bottines'],['Escarpin'],['lacets'],['Ballerines'],['sport'],['Bottes']\
          ,['Mocass'],['Tongs'],['Chaussons'],['Mules'],['scratch'],['Sacs'],['Espadrilles'],['Maroquinerie']]
creation(products,'ProductType',Label7)

'16 colonne(s) ont étaient ajoutées à la DataFrame'

##### *SupplierColor

In [46]:
Label8 = [['Rouge','Red','Bordeaux','rouge','red'],['Jaune','Yellow','jaune','yellow'],\
          ['Vert','vert','green','Green'],['Marine','Bleu','BlUE','Navy','bleu','blue'],\
          ['Noir','Black','Nero','noir','black','nero','Schwarz','NOIR'],\
          ['Blanc','Incolore','Beige','blanc','incolor','beige'],\
          ['Gris','Grey','Anthracite','gris','grey','anthracite'],\
          ['Marron','Brown','Taupe','Tan','Cognac','Chocolate','marron','brown','taupe','tan','cognac'],\
          ['Rose','Pink','rose','pink','purple','Purple']]

Label9 = [['Dark','dark','Fonc','fonc'],['Light','light','clair','Clair']]

creation(products,'SupplierColor',Label8)
creation(products,'SupplierColor',Label9)

'2 colonne(s) ont étaient ajoutées à la DataFrame'

##### *ProductColorId

In [47]:
# Faut le del y'a au maximum 20 commande pour un seul ProductColorId

##### *MinSize

In [48]:
Label10 = [35,36,37,38,39,40,41,42,43,200]
for i in (Label10):
    products['MinSize' + str(i)] = (products['MinSize'].eq(i)).astype('float')

products['MinSize' + '18' + '-' + '34'] = ((products['MinSize']>=18) & (products['MinSize']<=34)).astype('float')
products['MinSize' + '44' + '-' + '48'] = ((products['MinSize']>=44) & (products['MinSize']<=48)).astype('float')

##### *MaxSize

In [49]:
Label11 = [35,36,37,38,39,40,41,42,43,44,45,200]
for i in (Label11):
    products['MaxSize' + str(i)] = (products['MaxSize'].eq(i)).astype('float')

products['MaxSize' + '18' + '-' + '34'] = ((products['MaxSize']>=18) & (products['MaxSize']<=34)).astype('float')
products['MaxSize' + '46' + '-' + '48'] = ((products['MaxSize']>=46) & (products['MaxSize']<=48)).astype('float')

##### *CalfTurn

In [50]:
# Soit il y en a un soit non
Label12 = products['CalfTurn'].eq(0).astype('float')
products['CalfTurn'] = Label12

##### *UpperHeight

In [51]:
Label13 = [0,2,5,6,7,8,9,10,30]
for i in (Label13):
    products['UpperHeight' + str(i)] = (products['UpperHeight'].eq(i)).astype('float')

products['UpperHeight' + '31' + '-' + '38'] = ((products['UpperHeight']>=31) & (products['UpperHeight']<=38)).astype('float')

##### *HeelHeight

In [52]:
Label14 = [0,1,2,3,4,5,6,7,8,9,10]
for i in (Label14):
    products['HeelHeight' + str(i)] = (products['HeelHeight'].eq(i)).astype('float')

products['HeelHeightsup' + '11'] = ((products['HeelHeight']>=11)).astype('float')

##### *PurchasePriceHT

In [53]:
# Pas besoin de changer, c'est une relation d'ordre

##### *IsNewCollection

In [54]:
# Déja en Binaire

##### *SubtypeLabel

In [55]:
Label15 = [['rond'],['Basses'],['Montantes'],['scratch'],['lacet'],['bride'],['zip'],\
           ['pointu'],['compens'],['carr'],['ouvert'],['lastiques'],['Derbies'],['pluie'],\
           ['Bateau'],['Salom'],['Fourr'],['aiguille'],['boots'],['Spartiates'],['Richelieus'],\
           ['Babies']]
creation(products,'SubtypeLabel',Label15)

'22 colonne(s) ont étaient ajoutées à la DataFrame'

##### *UpperMaterialLabel

In [56]:
Label16 = [['Cuir'],['Nubuck'],['Textile'],['verni'],['Synth'],['Caoutchouc']]
creation(products,'UpperMaterialLabel',Label16)

'6 colonne(s) ont étaient ajoutées à la DataFrame'

##### *LiningMaterialLabel

In [57]:
Label17 = [['Textile'],['Cuir'],['Synth'],['doubl']]
creation(products,'LiningMaterialLabel',Label16)

'6 colonne(s) ont étaient ajoutées à la DataFrame'

##### *OutSoleMaterialLabel

In [58]:
Label18 = [['Gomme'],['Caoutchouc'],['Synth'],['Cuir']]
creation(products,'OutSoleMaterialLabel',Label17)

'4 colonne(s) ont étaient ajoutées à la DataFrame'

##### *RemovableSole

In [59]:
Label19 = products['RemovableSole'].eq(True).astype('float')
products['RemovableSole'] = Label19

In [60]:
products['RemovableSole'].value_counts()

0.0    444714
1.0     88806
Name: RemovableSole, dtype: int64

##### *SizeAdviceDescription

In [61]:
Label20 = [['chausse normalement','Prenez votre pointure habituelle'],['dessous'],['dessus'],['larges'],['fins']]
creation(products,'SizeAdviceDescription',Label20)

'5 colonne(s) ont étaient ajoutées à la DataFrame'

#### 4) Delete columns

In [62]:
#del products['VariantId'] # Necessaire pour merge orders/products
del products['GenderLabel']
del products['MarketTargetLabel']
del products['SeasonalityLabel']
del products ['UniverseLabel']
del products ['TypeBrand']
del products ['ProductId'] #environ 600 par productId
del products['ProductType']
del products['SupplierColor']
del products['ProductColorId'] #max 20 par productIdcolor
del products['MinSize']
del products['MaxSize']
del products['CalfTurn']
del products['UpperHeight']
del products['HeelHeight']
del products['SubtypeLabel']
del products['UpperMaterialLabel']
del products['LiningMaterialLabel']
del products['OutSoleMaterialLabel']
del products['SizeAdviceDescription']

# del products['BrandId']  # peut être a supprimer essayer de le traiter

In [63]:
products = products.fillna(0)

In [64]:
products.head()

,VariantId,SeasonLabel,BrandId,PurchasePriceHT,IsNewCollection,RemovableSole,Femme,Enfant,Homme,Sacs,...,Textile,verni,Synth,Caoutchouc,doubl,Prenez votre pointure habituelle,dessous,dessus,larges,fins
0,728257.0,1.0,66.0,30.50,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,806356.0,1.0,842.0,43.00,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,768790.0,1.0,988.0,54.90,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,515679.0,1.0,769.0,34.50,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1025246.0,1.0,1244.0,43.76,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


# MERGING

### Merge X_train

In [65]:
index = pd.DataFrame({'index':np.arange(X_train.shape[0])})
orders['index']=index # Evite les duplicata
y_train['index']=index

In [66]:
orders_products = pd.merge(orders,products,how='left',on='VariantId')
X_data = pd.merge(orders_products,customers,how='left',on='CustomerId')
data = pd.merge(y_train,X_data,how='left',on="index")
data = data.sort_values(by='index')
data.head()

,OrderNumber,LineItem,ReturnQuantity,ReturnQuantityBin,index,VariantId,CustomerId,IsoCode,Quantity,UnitPMPEUR,...,Age55-60,Sexe,FirstOrder-3Y,FirstOrder-4Y,FirstOrder-5Y,FirstOrder-6Y,FirstOrder-7Y,FirstOrder-8Y,FirstOrder-9Y,FirstOrder-10Y+
0,73521754,1,0.0,0.0,0,439729,12443972,FR,1,"52,64",...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,73521754,2,0.0,0.0,1,440174,12443972,FR,1,"52,64",...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,73525226,1,1.0,1.0,2,494501,12443958,FR,1,"13,17",...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,73529009,1,1.0,1.0,3,439590,12443946,FR,1,"56,4",...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,73529009,2,1.0,1.0,4,559476,12443946,FR,1,37,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [67]:
y_train.head()

,OrderNumber,LineItem,ReturnQuantity,ReturnQuantityBin,index
0,73521754,1,0.0,0.0,0
1,73521754,2,0.0,0.0,1
2,73525226,1,1.0,1.0,2
3,73529009,1,1.0,1.0,3
4,73529009,2,1.0,1.0,4


### Delete columns data

In [68]:
data.columns.tolist()

['OrderNumber',
 'LineItem',
 'ReturnQuantity',
 'ReturnQuantityBin',
 'index',
 'VariantId',
 'CustomerId',
 'IsoCode',
 'Quantity',
 'UnitPMPEUR',
 'OrderNumCustomer',
 'IsOnSale',
 'LineItem_1',
 'LineItem_2',
 'LineItem_3',
 'LineItem_4',
 'LineItem_5+',
 'OrderStatus',
 'OrderType',
 'Season',
 'maestro',
 'PayPal',
 'Gratuit',
 'iDeal',
 'que',
 'Bank',
 'DHL',
 'CustomerType',
 'Desktop',
 'Tablet',
 'Mobile',
 'Plein',
 'Soldes',
 'Avec CP',
 'Sans CP',
 'Vente Priv',
 'TotalLineItems_1',
 'TotalLineItems_2',
 'TotalLineItems_3',
 'TotalLineItems_4',
 'TotalLineItems_5',
 'TotalLineItems_6+',
 'Hour_21H_1H',
 'SeasonLabel',
 'BrandId',
 'PurchasePriceHT',
 'IsNewCollection',
 'RemovableSole',
 'Femme',
 'Enfant',
 'Homme',
 'Sacs',
 'Accessoires',
 'contemporain',
 'Suiveur',
 'Fashionistas',
 'Classique',
 'Confort',
 'Aventurier',
 'Skate',
 'Saisonnier',
 'Reconduit',
 'Permanent',
 'tente',
 'Ville',
 'Sport',
 'Green',
 'Luxe',
 'Soir',
 'Standard',
 'Super',
 'Baskets',
 

In [69]:
# del colonne for merging
del data['VariantId']
del data['CustomerId']
del data['OrderNumber']
del data['index']
del data['LineItem']

# Pour tester une premiere fois on supprime ces colonnes
#del data['IsoCode']
#del data['CountryISOCode']
#del data['BrandId']

### Delete columns X_data

In [70]:
X_data.columns.tolist()

['VariantId',
 'CustomerId',
 'IsoCode',
 'Quantity',
 'UnitPMPEUR',
 'OrderNumCustomer',
 'IsOnSale',
 'LineItem_1',
 'LineItem_2',
 'LineItem_3',
 'LineItem_4',
 'LineItem_5+',
 'OrderStatus',
 'OrderType',
 'Season',
 'maestro',
 'PayPal',
 'Gratuit',
 'iDeal',
 'que',
 'Bank',
 'DHL',
 'CustomerType',
 'Desktop',
 'Tablet',
 'Mobile',
 'Plein',
 'Soldes',
 'Avec CP',
 'Sans CP',
 'Vente Priv',
 'TotalLineItems_1',
 'TotalLineItems_2',
 'TotalLineItems_3',
 'TotalLineItems_4',
 'TotalLineItems_5',
 'TotalLineItems_6+',
 'Hour_21H_1H',
 'index',
 'SeasonLabel',
 'BrandId',
 'PurchasePriceHT',
 'IsNewCollection',
 'RemovableSole',
 'Femme',
 'Enfant',
 'Homme',
 'Sacs',
 'Accessoires',
 'contemporain',
 'Suiveur',
 'Fashionistas',
 'Classique',
 'Confort',
 'Aventurier',
 'Skate',
 'Saisonnier',
 'Reconduit',
 'Permanent',
 'tente',
 'Ville',
 'Sport',
 'Green',
 'Luxe',
 'Soir',
 'Standard',
 'Super',
 'Baskets',
 'Sandales',
 'Bottines',
 'Escarpin',
 'lacets',
 'Ballerines',
 'spor

In [71]:
# del colonne for merging
del X_data['VariantId']
del X_data['CustomerId']
del X_data['index']

# Pour tester une premiere fois on supprime ces colonnes
#del X_data['IsoCode']
#del X_data['CountryISOCode']
#del X_data['BrandId']

In [115]:
del X_data['UnitPMPEUR']
del X_data['IsoCode']
del X_data['BrandId']

In [72]:
X_data.head()

,IsoCode,Quantity,UnitPMPEUR,OrderNumCustomer,IsOnSale,LineItem_1,LineItem_2,LineItem_3,LineItem_4,LineItem_5+,...,Age55-60,Sexe,FirstOrder-3Y,FirstOrder-4Y,FirstOrder-5Y,FirstOrder-6Y,FirstOrder-7Y,FirstOrder-8Y,FirstOrder-9Y,FirstOrder-10Y+
0,FR,1,"52,64",1,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,FR,1,"52,64",1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,FR,1,"13,17",1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,FR,1,"56,4",1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,FR,1,37,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Preprocessing Test set

In [73]:
X_test.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,DeviceTypeLabel,PricingTypeLabel,TotalLineItems,Quantity,UnitPMPEUR,OrderCreationDate,OrderShipDate,OrderNumCustomer,IsOnSale,BillingPostalCode
0,90607166,754279,1,14369241,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Fidélisé,GB,Desktop,Promo Avec CP,2,1,"52,92",2014-12-02 14:59:54,2014-12-03 19:40:00,3,0.0,SW4 0DX
1,90621016,1043714,1,16121040,Expédié,DIRECT,Automne/Hiver,Klarna_Invoice,Fidélisé,DE,Desktop,Promo Avec CP,1,1,"32,4355",2014-12-02 15:00:41,2014-12-03 11:24:00,2,0.0,66887
2,90630628,302486,1,16271310,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,Desktop,Plein Tarif,1,1,"66,65",2014-12-02 15:01:18,2014-12-03 17:21:00,1,0.0,77600
3,90641377,1300698,1,16271314,Expédié,DIRECT,Automne/Hiver,PayPal,Nouveau,DE,Desktop,Promo Avec CP,1,1,"35,72",2014-12-02 15:03:24,2014-12-03 13:13:00,1,0.0,45879
4,90652456,1392714,1,16251703,Expédié,DIRECT,Automne/Hiver,BankTransfer_IBAN,Nouveau,ES,Desktop,Plein Tarif,1,1,"23,86",2014-12-02 15:04:13,2014-12-04 17:41:00,1,0.0,08006


### LineItem

In [74]:
for i in range(1,5):
    X_test['LineItem' + '_' + str(i) ] = (X_test['LineItem']==i).astype('float')
X_test['LineItem_5+'] = (X_test['LineItem']>=5).astype('float')

### OrderStatusLabel

In [75]:
X_test['OrderStatus'] = X_test['OrderStatusLabel'].eq('Expédié').astype('float')

### OrderTypelabel

In [76]:
X_test['OrderType'] = X_test['OrderTypelabel'].eq('DIRECT').astype('float')

### SeasonLabel

In [77]:
X_test['Season'] = X_test['SeasonLabel'].eq('Automne/Hiver').astype('float')

### PayementModeLabel

In [78]:
Label = [['Carte','maestro'],['PayPal'],['Gratuit'],['iDeal'],['que'],['Bank'],['DHL']]
creation(X_test,'PayementModeLabel',Label)

'7 colonne(s) ont étaient ajoutées à la DataFrame'

### CustomerTypeLabel

In [79]:
X_test['CustomerType'] = X_test['CustomerTypeLabel'].eq('Fidélisé').astype('float')

### DeviceTypeLabel

In [80]:
Label2 = [['Desktop'],['Tablet'],['Mobile']]
creation(X_test,'DeviceTypeLabel',Label2)

'3 colonne(s) ont étaient ajoutées à la DataFrame'

### PricingTypeLabel

In [81]:
Label3 = [['Plein'],['Soldes'],['Avec CP'],['Sans CP'],['Vente Priv']]
creation(X_test,'PricingTypeLabel',Label3)

'5 colonne(s) ont étaient ajoutées à la DataFrame'

### TotalLineItems

In [82]:
for i in range(1,6):
    X_test['TotalLineItems' + '_' + str(i) ] = (X_test['TotalLineItems']==i).astype('float')
X_test['TotalLineItems_6+'] = (X_test['TotalLineItems']>=6).astype('float')

### OrderCreationDate

In [83]:
X_test['OrderCreationDate'] = X_test['OrderCreationDate'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )

In [84]:
temp = X_test['OrderCreationDate'].apply(lambda x : x.hour)

X_test['Hour_21H_1H'] = ((temp<=1) | (temp>=21)).astype('float')

In [85]:
X_test.head()

,OrderNumber,VariantId,LineItem,CustomerId,OrderStatusLabel,OrderTypelabel,SeasonLabel,PayementModeLabel,CustomerTypeLabel,IsoCode,...,Avec CP,Sans CP,Vente Priv,TotalLineItems_1,TotalLineItems_2,TotalLineItems_3,TotalLineItems_4,TotalLineItems_5,TotalLineItems_6+,Hour_21H_1H
0,90607166,754279,1,14369241,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Fidélisé,GB,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,90621016,1043714,1,16121040,Expédié,DIRECT,Automne/Hiver,Klarna_Invoice,Fidélisé,DE,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,90630628,302486,1,16271310,Expédié,DIRECT,Automne/Hiver,Carte bancaire,Nouveau,FR,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,90641377,1300698,1,16271314,Expédié,DIRECT,Automne/Hiver,PayPal,Nouveau,DE,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,90652456,1392714,1,16251703,Expédié,DIRECT,Automne/Hiver,BankTransfer_IBAN,Nouveau,ES,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### Delete columns

In [86]:
# delete columns 
del X_test['OrderNumber']
#del X_test['VariantId'] # Necssaire pour merge orders/products
del X_test['LineItem']
#del X_test['CustomerId'] # Necessaire pour merge orders/customers
del X_test['OrderStatusLabel']
del X_test['OrderTypelabel']
del X_test['SeasonLabel']
del X_test['PayementModeLabel']
del X_test['CustomerTypeLabel']
del X_test['DeviceTypeLabel']
del X_test['PricingTypeLabel']
del X_test['TotalLineItems']
del X_test['OrderCreationDate']
del X_test['OrderShipDate']
del X_test['BillingPostalCode']
# del X_test['OrderNumCustomer']  # Le garder et voir statistiques dessus, pas traité.

In [87]:
X_test = X_test.fillna(0)

In [88]:
X_test.head()

,VariantId,CustomerId,IsoCode,Quantity,UnitPMPEUR,OrderNumCustomer,IsOnSale,LineItem_1,LineItem_2,LineItem_3,...,Avec CP,Sans CP,Vente Priv,TotalLineItems_1,TotalLineItems_2,TotalLineItems_3,TotalLineItems_4,TotalLineItems_5,TotalLineItems_6+,Hour_21H_1H
0,754279,14369241,GB,1,"52,92",3,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1043714,16121040,DE,1,"32,4355",2,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,302486,16271310,FR,1,"66,65",1,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1300698,16271314,DE,1,"35,72",1,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1392714,16251703,ES,1,"23,86",1,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### Merge X_test

In [89]:
test_products = pd.merge(X_test,products,how='left',on='VariantId')
test_data = pd.merge(test_products,customers,how='left',on='CustomerId')
test_data.head()

,VariantId,CustomerId,IsoCode,Quantity,UnitPMPEUR,OrderNumCustomer,IsOnSale,LineItem_1,LineItem_2,LineItem_3,...,Age55-60,Sexe,FirstOrder-3Y,FirstOrder-4Y,FirstOrder-5Y,FirstOrder-6Y,FirstOrder-7Y,FirstOrder-8Y,FirstOrder-9Y,FirstOrder-10Y+
0,754279,14369241,GB,1,"52,92",3,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1043714,16121040,DE,1,"32,4355",2,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,302486,16271310,FR,1,"66,65",1,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1300698,16271314,DE,1,"35,72",1,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1392714,16251703,ES,1,"23,86",1,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### Delete columns test_data

In [90]:
# del colonne for merging
del test_data['VariantId']
del test_data['CustomerId']
#del test_data['OrderNumber']

# Pour tester une premiere fois on supprime ces colonnes
#del test_data['IsoCode']
#del test_data['CountryISOCode']
#del test_data['BrandId']

In [91]:
test_data.head()

,IsoCode,Quantity,UnitPMPEUR,OrderNumCustomer,IsOnSale,LineItem_1,LineItem_2,LineItem_3,LineItem_4,LineItem_5+,...,Age55-60,Sexe,FirstOrder-3Y,FirstOrder-4Y,FirstOrder-5Y,FirstOrder-6Y,FirstOrder-7Y,FirstOrder-8Y,FirstOrder-9Y,FirstOrder-10Y+
0,GB,1,"52,92",3,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,DE,1,"32,4355",2,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,FR,1,"66,65",1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,DE,1,"35,72",1,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ES,1,"23,86",1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
liste_test = test_data.columns.tolist()
liste_train = X_data.columns.tolist()

187
187


In [114]:
del test_data['BrandId']
del test_data['UnitPMPEUR']
del test_data['IsoCode']

# Standardize the data

In [92]:
#data.UnitPMPEUR

# Defining a feature transformation

In [119]:
def funk_mask(d,masked_features):
    " Defining a simple mask over the input data "
    columns_ext = masked_features
    X1 = d.loc[:,[xx for xx in d.columns if xx not in columns_ext]]
    print("1")
    g = lambda x: x.replace(",",".")
    print("2")
    #X1.UnitPMPEUR = map(np.float64,(map(g,X1.UnitPMPEUR)))
    print("3")
    columns2bin = [x for x in X1.columns if X1[x].dtype == np.dtype('O')]
    print("4")
    X2 = pd.get_dummies(X1.loc[:,columns2bin])
    print("5")
    X1 = X1.loc[:,[xx for xx in X1.columns if xx not in columns2bin]]
    print("6")
    res = pd.concat([X1,X2],axis=1)
    print("7")
    res = res.fillna(0)
    return(res)

# Applying the mask

In [117]:
features = []

In [120]:
x1 = funk_mask(X_data,features)
x2 = funk_mask(test_data,features)
seleckt_columns = np.intersect1d(x1.columns,x2.columns)
x1 = x1.loc[:,seleckt_columns]
x2 = x2.loc[:,seleckt_columns]

1
2
3
4
5
6
7
1
2
3
4
5
6
7


# Standardizing the Data

In [121]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(x1)

# Apply transform to both the training set and the test set.
x1 = scaler.transform(x1)
x2 = scaler.transform(x2)

# PCA

In [105]:
from sklearn.decomposition import PCA

In [122]:
# Make an instance of the Model
pca = PCA(n_components=5)

# Fit PCA on training set
pca.fit(x1)

# Apply the mapping (transform) to both the training set and the test set
x1 = pca.transform(x1)
x2 = pca.transform(x2)

# Supervised learning : Logistic regression model

In [107]:
clf = RandomForestClassifier()
clf.fit(x1[:100000], y_train.ReturnQuantityBin[:100000])
y_tosubmit = clf.predict_proba(x2)
score_forest = np.mean(cross_val_score(clf,x1[:100000], y_train.ReturnQuantityBin[:100000],cv=10))
print(score_forest)

0.761779947052


In [123]:
np.savetxt('y_pred.txt', y_tosubmit[:,1], fmt='%f')

# Score of our prediction : on the train

In [124]:
yres = clf.predict_proba(x1[:100001])
print(roc_auc_score(y_train.ReturnQuantityBin.iloc[:100001],yres[:,1]))

# Submission to the system
np.savetxt('y_pred.txt', y_tosubmit[:,1], fmt='%f')

0.475151799557
